In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import joblib
emb_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipol_embedd_7410_128_768_alberto_training.joblib")
wmal_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipolc_wmal_training_7410_128.joblib")

In [ ]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/sentipolc_train_processed.csv")

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["oneg"]
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 0]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]
[1]


In [ ]:
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate
!pip install keras-self-attention

In [ ]:
!pip install keras-self-attention
from keras_self_attention import SeqWeightedAttention, SeqSelfAttention
from keras.layers import Add, Multiply

In [ ]:
import keras
input_emb = Input(shape=(128,768))
input_wmal = Input(shape=(128,))

#Bi-LSTM
#bi = Bidirectional(LSTM(32, return_sequences=True))(input_emb)
bi = LSTM(64, return_sequences=True)(input_emb)
#att = SeqWeightedAttention()(bi)
att = SeqSelfAttention(attention_width=15,attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       attention_activation="sigmoid",
                       kernel_regularizer=keras.regularizers.l2(1e-8),#(1e-6),
                       bias_regularizer=keras.regularizers.l1(1e-8),#(1e-4),
                       attention_regularizer_weight=1e-8,#1e-4,
                       use_attention_bias=True,return_attention=False) (bi)

app = Multiply() ([bi, att])                      
flat = Flatten()(app)

#WMAL 
d1 = Dense(64) (input_wmal)

#Concat
concat = Concatenate(axis=1)([flat, d1])

d2 = Dense(32) (concat)
output = Dense(2,activation="softmax") (d2)

m = keras.Model(inputs=[input_emb,input_wmal], outputs=output)

opt = keras.optimizers.Adam(learning_rate=0.00005)
m.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])


In [ ]:
m.summary()

In [ ]:
#m.fit([emb_matrix_train,wmal_matrix_train],onehot_encoded,batch_size=128, epochs=10,validation_split=0.15)
m.load_weights("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_3_sentipolc/model_3_pos_sentipolc_0.73645.h5")

In [ ]:
import joblib
import pandas as pd
emb_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/creagritrend_embedd_986_128_768_alberto.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/creagritrend_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/creagritrend_wmal_agritrend_986_128.joblib")

In [ ]:
# define example
data = dataset_test["label_pos"]
values = array(data)
print(values)
# integer encode
integer_encoded_test = label_encoder.transform(values)
print(integer_encoded_test)
# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)
print(onehot_encoded_test)

In [ ]:
predictions = m.predict([emb_matrix_test,wmal_matrix_test])

In [ ]:
print(predictions)

[[1.0000000e+00 2.2970085e-08]
 [9.9982798e-01 1.7202580e-04]
 [9.9995875e-01 4.1254432e-05]
 ...
 [9.9762517e-01 2.3747978e-03]
 [9.9999917e-01 8.2827910e-07]
 [1.0000000e+00 8.8940801e-09]]


In [ ]:
import numpy as np
predictions_vec = np.zeros((986,4))

In [ ]:
i = 0
confidences= []
for item in predictions:
  index = np.argmax(item);
  confidences.append(item[index])
  predictions_vec[i][index] = 1
  i = i+1

In [ ]:
labels_to_eval = []
for vec in predictions_vec:
  inverted = label_encoder.inverse_transform([argmax(vec)])[0]
  labels_to_eval.append(inverted)

In [ ]:
confidences

In [ ]:
len(labels_to_eval)

986

In [ ]:
dataset_test["predicted_label_pos"] = labels_to_eval
dataset_test["confidence_label_pos"] = confidences

In [ ]:
!pip install scikit-learn

from sklearn.metrics import classification_report
print(classification_report(dataset_test["label_pos"],labels_to_eval,digits=5))

              precision    recall  f1-score   support

           0    0.34111   0.97771   0.50577       314
           1    0.91860   0.11756   0.20844       672

    accuracy                        0.39148       986
   macro avg    0.62986   0.54763   0.35710       986
weighted avg    0.73470   0.39148   0.30313       986



In [ ]:
dataset_test

In [ ]:
#m.save("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_1_neg_agritrend_0.57629.h5")

In [ ]:
dataset_test.to_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/predictions_AGRITREND_pos_full.csv");